# The goal of this program is to create a data frame that includes:
## Champion picks of every current Challenger player (including their climb games) **from season start**

In [4]:
import pandas as pd
import os
import cassiopeia as cass
import requests
import random
from ratelimit import limits
import json
import datetime
pwd = os.getcwd()
pd.options.display.max_rows = 10
cass.set_riot_api_key("REDACTED")

key = 'REDACTED'

### Getting used to using Champion IDs & Returning Values into Arrrays

In [5]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives, region="NA")

summoner_name = []

for item in challenger_league:
    summoner_name.append(item.summoner.name)

summoner_name


['C9 Winsome FAN',
 'Dacryagogue',
 'Nikkone',
 'lafyèv',
 'Szochs',
 'Prymari',
 'Mr Bluehat',
 'MaIthaeI',
 'Nimdle',
 'TwTv imsudzzi',
 'Nim Chimpsky',
 'Ralyk',
 'Ethiridis',
 'Le Kachu',
 'Anotherhope',
 'Peruvian GAAP',
 'DrewDozer',
 'Kurose',
 'CarryingPB',
 'Alien Hominid',
 'Protos',
 'SwiFtSluG',
 'Veerchand',
 'GOOD LUCK SOL',
 'yticanet001',
 'Chadmaister',
 'nifty jg',
 'yukino cat',
 'cosha2',
 'Int Overflow',
 'admirable potato',
 'Bashful Iceberg',
 'MUSTACHEmanFIGHT',
 'I will trade',
 'rocks908',
 'kai fú',
 'Feather Daddy',
 '21 Stylish',
 'seal on top',
 'Pomerzz Dongerzz',
 'Mobility',
 'zhe bu he li',
 'Eyla',
 'WX Rika19981209',
 'Zeri247',
 'Smoothie',
 'Denathor',
 'CLG Luger',
 'COOKIEMONSTER123',
 'Pseudo',
 '5kid0',
 'rklckp251',
 'corejj cat',
 'winstxn',
 'elodragondotcom',
 'KingArtLeywin',
 'lunacia',
 'Yorushik4',
 'no invade pls',
 'hoidro',
 'itdoesntmatter',
 'VX S9hua9',
 'Jauny12',
 'soupfan4',
 'zhong john xina',
 'Spìrit',
 '45tbng90s8r4j1',
 'N

In [6]:
summoner_puuid = []  

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    summoner_puuid.append(bucket.puuid)
    
summoner_puuid


Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/M_wuM7wfU9z_Y7itdVdgXK3RtIgokeAHgpepsDZIQiVewVNg9FU5GX_2Cg
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/g8XOuiUMObKtgybuF3teLgprgodlIciHqvC_VtDLRrQ--G0
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/IH55McgQIu3XL1JHEixp7VzHOIgr8qcmEcttZVlE_zuT-Dc
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/LcYUy7_-fotfm0MPPjlEHRwTNM9bx0tVcbNRGo4E4Zz4_po
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/eYoHKWFioSmHP-IIJyLShUeEI5wHDwlt9p4qemBpddw4IPA
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/I94WZq_9XbswEALgKguZOERlqUkf5mqG9w-tD9zLm9rwLhA
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/OTItgmZvYRx5vs0eIkyD-7y3z1t5tEXtsvnNI9cOA79zJTU
Making call: https://na1.api.riotgames.com/lol/summoner/v4/summoners/4p-q-xhxRhsf0ld_PZMozwTRGjVTCdTu5l0Mgd6Fp_ptvTU
Making call: https://na1.api.riotgames.com/lol/summon

['aKl91xjcd3Va6WcRstEo9m91FczG3Z_32bRQD_iDonw5wjoK762WdK_AYkSuwyhor4zkB1W5uJrvDQ',
 'Y59Ic22ufTqYSM9B3_aTDKIR06iULHVhE-0Yqni6UiLRVkb7run_3P51mSB-qOxzx1_hxWwxpzz3rw',
 'U0CCKR3p72EqEZTetkqpPaK0kWsABq5C58PL-dBE_8e2uuF94svoj-0GKwjABOM4RHJbuDKmST_Fvw',
 'Aa-hklr-tjePa8QHL57jkBh2K3eVnzjI6HEskflay9AvCU6eIjCUmQcr5IqtaweNJW4DPNtvLwYgVw',
 'sVcMirnYf1PXcHytcW00sEGsMOmFjY-jZstLLepvwFt1N5gnz8dUQ1mE809SKAiPla7ZDcB_aSR5Lw',
 'dX0-0N-baFw2uLu1vSyrP9ybRNAMJ764TXaq4JLhuRemCITa2v-U-UWwtbpc4gFn0DOsfvcoZ_LaQA',
 'vVPBB6nUFThmeZKvHi8_u8nikIEw4Q3AmODk66twrGTx76xAjlf1pXVoSjA1UZmM6FIXguZV7wNLxA',
 'NVjMCw5a9ae-EVIzdydQGX85dnnf4gzzcnVogYY6YnXJEUlps3tHPyJFXTQl1Ya-8oF81mQkULZvqg',
 'alBkBWzbDpcBTfnkizHIUdW9RD1SsD-wiTNyiq5CD_BDeqQ4Qv_uZdaW1d2ZbEp1scjXYJbYJ68ryw',
 'JTV_LIB5o3gRyTn45AWOK5DKxKrggNMInUxT_4Yd5xjG4XgADvwFT0SORGiyGYVTZwYZdIKcGJbudQ',
 'a4jTKlyi3h1jVindjxNxp_0iOH2-PsAvecN-TgDbbkKWXmGaBdPbDYXbjjd6xz5xIcNTIAXhATsX_w',
 'P99kDqtzctlvX4v07Gd-o87Ap3JL5c5apN_mJLhz6dnUPfJYZZjV0dGb53a9y5RiaNAEpiaPfUHAWA',
 'Eu

## Now that I have a list of summoner names and puuids, I can start pulling match histories

In [7]:
total_match_list = []

for item in summoner_name:
    bucket = cass.Summoner(name=item, region='NA')
    challenger_history = (cass.get_match_history(region=cass.data.Region.north_america,
                                             queue=cass.data.Queue.ranked_solo_fives, puuid=bucket.puuid,
                                             continent=cass.data.Continent.americas))
    total_match_list.append(challenger_history)


total_match_list

Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/aKl91xjcd3Va6WcRstEo9m91FczG3Z_32bRQD_iDonw5wjoK762WdK_AYkSuwyhor4zkB1W5uJrvDQ/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/Y59Ic22ufTqYSM9B3_aTDKIR06iULHVhE-0Yqni6UiLRVkb7run_3P51mSB-qOxzx1_hxWwxpzz3rw/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/U0CCKR3p72EqEZTetkqpPaK0kWsABq5C58PL-dBE_8e2uuF94svoj-0GKwjABOM4RHJbuDKmST_Fvw/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/Aa-hklr-tjePa8QHL57jkBh2K3eVnzjI6HEskflay9AvCU6eIjCUmQcr5IqtaweNJW4DPNtvLwYgVw/ids?beginIndex=0&endIndex=100&queue=420
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/sVcMirnYf1PXcHytcW00sEGsMOmFjY-jZstLLepvwFt1N5gnz8dUQ1mE809SKAiPla7ZDcB_aSR5Lw/ids?beginIndex=0&endIndex=100&queue=420
Making call: ht

[[<cassiopeia.core.match.Match at 0x239a07254f0>,

In [8]:

champion_picks = []
data = []
matches = []

for position, match_list in enumerate(total_match_list):
    bucket = cass.Summoner(name=summoner_name[position], region='NA')
    for match in match_list:
        champion = match.participants[bucket].champion.name
        champion_picks.append(champion)
        matches.append(match)
        tup = (bucket.name, champion, match)
        data.append(tup)

champion_picks
data        
matches

        


Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4223232701
Making call: https://cdn.merakianalytics.com/riot/lol/resources/patches.json
Making call: https://ddragon.leagueoflegends.com/api/versions.json
Making call: https://ddragon.leagueoflegends.com/cdn/12.4.1/data/en_US/championFull.json
Making call: https://ddragon.leagueoflegends.com/realms/na.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209455860
Making call: https://ddragon.leagueoflegends.com/cdn/12.3.1/data/en_US/championFull.json
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209278526
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209231294
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4209124261
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4208977212
Making call: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4208829612
Making call: https://ameri

 ...]

In [9]:
data.sort

data

[('C9 Winsome FAN', 'Bard', <cassiopeia.core.match.Match at 0x239a07254f0>),
 ('C9 Winsome FAN', 'Poppy', <cassiopeia.core.match.Match at 0x239a0725550>),
 ('C9 Winsome FAN', 'Sona', <cassiopeia.core.match.Match at 0x239a07259d0>),
 ('C9 Winsome FAN', 'Lulu', <cassiopeia.core.match.Match at 0x239a0725ac0>),
 ('C9 Winsome FAN', 'Senna', <cassiopeia.core.match.Match at 0x239a0725be0>),
 ('C9 Winsome FAN',
  'Nautilus',
  <cassiopeia.core.match.Match at 0x239a0725d00>),
 ('C9 Winsome FAN', 'Thresh', <cassiopeia.core.match.Match at 0x239a0725e20>),
 ('C9 Winsome FAN', 'Senna', <cassiopeia.core.match.Match at 0x239a0725f40>),
 ('C9 Winsome FAN', 'Poppy', <cassiopeia.core.match.Match at 0x239a07270a0>),
 ('C9 Winsome FAN', 'Senna', <cassiopeia.core.match.Match at 0x239a07271c0>),
 ('C9 Winsome FAN', 'Janna', <cassiopeia.core.match.Match at 0x239a07272e0>),
 ('C9 Winsome FAN', 'Thresh', <cassiopeia.core.match.Match at 0x239a0727400>),
 ('C9 Winsome FAN', 'Lulu', <cassiopeia.core.match.Match a

In [10]:
dataframe = pd.DataFrame(data, columns= ['Summoner', 'Champion', 'Match'])

dataframe

,Summoner,Champion,Match
0,C9 Winsome FAN,Bard,"Match(id=4223232701, region='AMERICAS')"
1,C9 Winsome FAN,Poppy,"Match(id=4209455860, region='AMERICAS')"
2,C9 Winsome FAN,Sona,"Match(id=4209278526, region='AMERICAS')"
3,C9 Winsome FAN,Lulu,"Match(id=4209231294, region='AMERICAS')"
4,C9 Winsome FAN,Senna,"Match(id=4209124261, region='AMERICAS')"
...,...,...,...
5995,DEMON BRAND ARC,Syndra,"Match(id=4223334302, region='AMERICAS')"
5996,DEMON BRAND ARC,Syndra,"Match(id=4223320564, region='AMERICAS')"
5997,DEMON BRAND ARC,Tristana,"Match(id=4223272448, region='AMERICAS')"
5998,DEMON BRAND ARC,Tristana,"Match(id=4223145817, region='AMERICAS')"


## I now know how pull matchlists, sort them by summoner, and store them into a data frame

### Now I need to figure out how to see which champions have the most picks, and test the things like:

#### "Most one trick", "Least one trick", "Highest Pick Rate for Climb", "Lowest Pick Rate for Climb" etc...

In [11]:
dataframe

dataframe_modified = dataframe.copy()

matches_set = set(matches)

match_count = len(matches_set)

match_count


2881

In [12]:
dataframe_modified.describe()

,Summoner,Champion,Match
count,6000,6000,6000
unique,300,155,2881
top,C9 Winsome FAN,Jinx,"Match(id=4223272448, region='AMERICAS')"
freq,20,200,10


# Most Picked Champions

In [13]:
from collections import Counter

In [14]:
champ_pick_num = Counter(elem[1] for elem in data)

sorted_10  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=True)[:10]

sorted_10

for item in sorted_10:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Jinx: picked in 6.942034015966677% of games

Graves: picked in 5.380076362374176% of games

Aphelios: picked in 5.206525511975009% of games

Ahri: picked in 5.206525511975009% of games

Jhin: picked in 5.171815341895175% of games

Gwen: picked in 4.720583130857341% of games

Zeri: picked in 4.19993057965984% of games

Viego: picked in 4.026379729260674% of games

Ezreal: picked in 3.9569593891010064% of games

Lee Sin: picked in 3.713988198542173% of games



# Least Picked Champions 
##### *With at least one pick*

In [15]:
sorted_10_bottom  = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)[:10]

sorted_10_bottom

for item in sorted_10_bottom:
    print(item + ": picked in " + str(100*((champ_pick_num[item]/(match_count))))  +"% of games\n")

Nocturne: picked in 0.03471017007983339% of games

Vel'Koz: picked in 0.03471017007983339% of games

Azir: picked in 0.03471017007983339% of games

Master Yi: picked in 0.03471017007983339% of games

Skarner: picked in 0.03471017007983339% of games

Warwick: picked in 0.03471017007983339% of games

Rammus: picked in 0.03471017007983339% of games

Nasus: picked in 0.06942034015966678% of games

Kled: picked in 0.06942034015966678% of games

Illaoi: picked in 0.06942034015966678% of games



# Champions Not Picked at All

In [16]:
picked_champs = sorted(champ_pick_num, key=champ_pick_num.get, reverse=False)

full_champ_list = []

list = cass.get_champions(region='NA')

for item in list:
    full_champ_list.append(item.name)

champions_not_picked = set(full_champ_list).difference(set(picked_champs))


print("These Champions were picked in 0 games:") 
for item in champions_not_picked:
    print('\n' + item)

These Champions were picked in 0 games:

Yorick

Singed

Anivia

Taric


# Most one-trick player

In [17]:
# How do I get the mode with my current tools [total_match_list, ('Summoner', 'Champion', 'Match'), summoner_name]
# My current plan is to use a dataframe to get the mode for each Summoner, I can also use the dataframe to get the total matches played per summoner. 

# ^This idea was good in my head but is total dogshit. I think I need to create some function to iterate through the list of tuples, and group them under their summoner name
#  (MAYBE THIS WAS SOMETHING THAT I SHOULD HAVE DONE DURING THE DATA COLLECTION PROCESS BUT HERE WE ARE, WE'RE TRYING TO LEARN EVEN IF ITS NOT OPTIMAL)



# I could then create a list of tuples (Player, Champion, Mode) 
# Not sure what to do afterward

# Perhaps create a parallel list of modes, and use a max function to pull the string, and then use that string to pull the Summoner and Champion index from the tuple

In [42]:
picks_aggregated_to_summoner = {}

for name in summoner_name:
    history_list = []
    for tuple in data:
        if name == tuple[0]:
            history_list.append(tuple[1])
    picks_aggregated_to_summoner.update({(int(summoner_name.index(str(name)))): history_list})


picks_aggregated_to_summoner

{0: ['Bard',
  'Poppy',
  'Sona',
  'Lulu',
  'Senna',
  'Nautilus',
  'Thresh',
  'Senna',
  'Poppy',
  'Senna',
  'Janna',
  'Thresh',
  'Lulu',
  'Senna',
  'Lux',
  'Karma',
  'Lulu',
  'Pyke',
  'Gragas',
  'Seraphine'],
 1: ['Karma',
  'Nautilus',
  'Renata Glasc',
  'Thresh',
  'Leona',
  'Rakan',
  'Lulu',
  'Lulu',
  'Karma',
  'Nautilus',
  'Nautilus',
  'Nautilus',
  'Nautilus',
  'Soraka',
  'Leona',
  'Caitlyn',
  'Thresh',
  "Kai'Sa",
  'Yuumi',
  'Brand'],
 2: ['Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks',
  'Fiddlesticks'],
 3: ["Kog'Maw",
  'Akali',
  'Zeri',
  'Jinx',
  'Aphelios',
  'Orianna',
  'Aphelios',
  'Akali',
  'Zeri',
  'Karma',
  'Akali',
  'Nautilus',
  'Karma',
  'S

In [1]:
# now we can call the modes from each player in the dictionary, divide them by the length of the list we got them from, and then place those percentages into a list.
# we can use the index value of the maximum value in that list to return the player name from the summoner_name list
# this process can also be done to figure out who played the most games

import statistics 

mode_tup_list = []
modes = []
max_moders = []


for key, list in picks_aggregated_to_summoner.items():
    mode_champion = statistics.mode(list)
    mode_value = list.count(mode_champion)
    pick_percentage = (mode_value / len(list)) * 100
    mode_tup_list.append((summoner_name[key], mode_champion, mode_value, pick_percentage))
    modes.append(str(mode_value))

mode_tup_list

largest_one_trick_value = max(pick_percentage)

for i, mode, in enumerate(modes):
    if mode == largest_one_trick_value:
        location = i
        max_moders.append((summoner_name[location], pick_percentage))

max_moders



NameError: name 'picks_aggregated_to_summoner' is not defined

In [67]:
modes

['4',
 '5',
 '20',
 '4',
 '7',
 '4',
 '16',
 '20',
 '15',
 '5',
 '18',
 '11',
 '10',
 '4',
 '12',
 '4',
 '6',
 '7',
 '12',
 '4',
 '13',
 '18',
 '17',
 '9',
 '2',
 '3',
 '4',
 '4',
 '8',
 '14',
 '5',
 '9',
 '5',
 '4',
 '8',
 '8',
 '19',
 '19',
 '11',
 '12',
 '8',
 '13',
 '4',
 '4',
 '4',
 '4',
 '12',
 '4',
 '7',
 '5',
 '6',
 '5',
 '6',
 '5',
 '9',
 '18',
 '11',
 '2',
 '11',
 '16',
 '4',
 '18',
 '5',
 '7',
 '14',
 '4',
 '19',
 '2',
 '14',
 '5',
 '10',
 '5',
 '4',
 '18',
 '4',
 '11',
 '5',
 '19',
 '5',
 '6',
 '5',
 '10',
 '10',
 '3',
 '3',
 '10',
 '7',
 '5',
 '5',
 '4',
 '4',
 '16',
 '9',
 '4',
 '5',
 '8',
 '9',
 '6',
 '4',
 '3',
 '5',
 '6',
 '11',
 '12',
 '7',
 '5',
 '5',
 '11',
 '4',
 '8',
 '4',
 '8',
 '5',
 '10',
 '11',
 '9',
 '4',
 '6',
 '4',
 '13',
 '8',
 '14',
 '9',
 '6',
 '13',
 '6',
 '4',
 '5',
 '4',
 '5',
 '10',
 '5',
 '13',
 '4',
 '2',
 '3',
 '8',
 '18',
 '4',
 '7',
 '6',
 '7',
 '8',
 '5',
 '7',
 '18',
 '4',
 '4',
 '15',
 '4',
 '4',
 '5',
 '7',
 '4',
 '7',
 '2',
 '5',
 '12',
 '6

# Least one-trick player



'rocks908'

# Least Games Played


TypeError: 'type' object is not subscriptable

# Most Games Played
